In [7]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import atan2, log, pi
import datetime as dt

In [8]:
def read_saildrone(filename_usv):
    #filename_usv='F:/data/cruise_data/saildrone/baja-2018/saildrone-gen_4-baja_2018-EP-sd1002-ALL-1_min-v1.nc'
    ds_usv=xr.open_dataset(filename_usv)
    dataset=ds_usv
    #    ds_usv = xr.open_dataset(filename_usv)
#    dataset=xr.open_dataset(filename_usv,decode_times=False)
    ilen_usv=(len(ds_usv.LAT['obs']))
    #dates_usv64=ds_usv.TIME[0,:].values
    #dates_usv=pd.to_datetime(dates_usv64, unit='ns')  #new way to caluclate the time array

    COORD_ATTR = 'time'
#    COORD_ATTR = 'lat lon time'

    varrays = {}

    for v in dataset.data_vars:
        # set attributes
        if v in ['TIME']:
#        if v in ['LAT', 'LON', 'TIME']:
            continue
        vattrs = dataset[v].attrs.copy()
        vattrs["coordinates"] = COORD_ATTR
        vattrs["_FillValue"] = np.nan #FLOAT_FILLVALUE

        values = np.ma.fix_invalid(dataset[v].values[0,:])
        if v == 'TEMP_CTD_MEAN':
            # convert SST to Kelvin
            values += 273.15

        # create new data array
        varrays[v] = xr.DataArray(
            values.filled(np.nan).astype(np.float32,copy=False),
            dims=['time'],
            attrs=vattrs
        )

    vattrs = dataset.UWND_MEAN.attrs.copy()
    vattrs['standard_name'] = 'wind_speed'
    vattrs['long_name'] = 'wind_speed'
    vattrs['valid_min'] = 0
    vattrs['valid_max'] = 100
    vattrs["_FillValue"] = np.nan #FLOAT_FILLVALUE
    vattrs['source'] = 'anemometer'
    vattrs['comment'] = 'Instrument located at to of Saildrone mast at ' +\
                    str(dataset.UWND_MEAN.installed_height)+' m' + '.  This was adjusted ' +\
                    'to 10 m as ws_10m = ws*log(10./1e-4))/log(WS_height/1e-4'
    vattrs['height'] = '10 m' #str(str(dataset.UWND_MEAN.installed_height)+' m')
    vattrs["coordinates"] = COORD_ATTR
    WS=(dataset['UWND_MEAN'].values[0,:]**2 + dataset['VWND_MEAN'].values[0,:]**2)**.5
    WS_height=int(dataset.UWND_MEAN.installed_height)
    WS_10m = (WS*log(10./1e-4))/log(WS_height/1e-4)
    varrays['wind_speed'] = xr.DataArray(
        WS_10m.astype(np.float32,copy=False),
        dims=['time'],
        attrs=vattrs
        )
    vattrs = dataset.UWND_MEAN.attrs.copy()
    vattrs['standard_name'] = 'wind_to_direction'
    vattrs['long_name'] = 'local wind direction'
    vattrs['valid_min'] = 0
    vattrs['valid_max'] = 360
    vattrs["_FillValue"] = np.nan #FLOAT_FILLVALUE
    vattrs['units'] = 'degrees'
    vattrs['source'] = 'anemometer'
    vattrs['height'] = str(str(dataset.UWND_MEAN.installed_height)+' m')
    vattrs["coordinates"] = COORD_ATTR
    WD=np.arctan2(dataset.VWND_MEAN.values[0,:], dataset.UWND_MEAN.values[0,:])*180/pi
    varrays['wind_direction'] = xr.DataArray(
        WD.astype(np.float32,copy=False),
        dims=['time'],
        attrs=vattrs
        )
    vattrs = dataset.TIME.attrs.copy()
    vattrs["_FillValue"] = np.nan #FLOAT_FILLVALUE
    values = ds_usv.TIME.values[0,:] 
    vtime = xr.DataArray(values,dims=['time'],attrs=vattrs)
    gattrs = dataset.attrs.copy()

    l3r = xr.Dataset(
        varrays,
        coords = {
            'time': vtime # ,'lat':dataset.LAT.values[0,:],'lon':dataset.LON.values[0,:]
#            'time': vtime ,'lat':dataset.LAT.values[0,:],'lon':dataset.LON.values[0,:]
        },
        attrs=gattrs
    )
    return l3r
    #l3r.to_netcdf('F:/data/cruise_data/saildrone/baja-2018/test.nc')
    #l3r=xr.open_dataset('F:/data/cruise_data/saildrone/baja-2018/test.nc')

In [12]:
#read in high resolution wind buoy data (10 minute)
sbuoy_id='46011'
sdep='D5'
fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201804_'+sdep+'_v0010min_xrformat.nc'
xr_buoy10=xr.open_dataset(fname)
fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201805_'+sdep+'_v0010min_xrformat.nc'
tem=xr.open_dataset(fname)
xr_buoy10 = xr.concat([xr_buoy10, tem],'time')
fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201806_'+sdep+'_v0010min_xrformat.nc'
tem=xr.open_dataset(fname)
xr_buoy10 = xr.concat([xr_buoy10, tem],'time')
time10=xr_buoy10.time#read in hourly buoy data
fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201804_'+sdep+'_v00hrly_xrformat.nc'
xr_buoy=xr.open_dataset(fname)
fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201805_'+sdep+'_v00hrly_xrformat.nc'
tem=xr.open_dataset(fname)
xr_buoy = xr.concat([xr_buoy, tem],'time')
fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201806_'+sdep+'_v00hrly_xrformat.nc'
tem=xr.open_dataset(fname)
xr_buoy = xr.concat([xr_buoy, tem],'time')
time=xr_buoy.time
#fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_46011_201804_D5_v00hrly_xrformat.nc'
#xr_buoy=xr.open_dataset(fname)
cond = xr_buoy.lat > 90
xr_buoy.lat[cond]=np.nan
cond = xr_buoy.lon > 90
xr_buoy.lon[cond]=np.nan
blat=xr_buoy.lat.mean()
blon=xr_buoy.lon.mean()
btime=xr_buoy.time


In [13]:
xr_buoy

<xarray.Dataset>
Dimensions:                (time: 2237)
Coordinates:
  * time                   (time) datetime64[ns] 2018-04-01T00:50:00 ... 2018-06-30T23:50:00
Data variables:
    sst1                   (time) float32 287.25 287.25 287.25 ... nan 289.45
    sst1qc                 (time) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 -9 0
    wnd1                   (time) float32 6.0 5.8 4.8 ... 2.1 9.96921e+36 1.3
    wnd1qc                 (time) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 -9 0
    wdir1                  (time) int16 315 321 310 315 ... 252 272 -32767 273
    wdir1qc                (time) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 -9 0
    wnd2                   (time) float32 3.9 3.5 1.4 ... 0.0 9.96921e+36 0.0
    wnd2qc                 (time) int8 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 -9 2
    wdir2                  (time) int16 275 246 50 228 ... 193 193 -32767 193
    wdir2qc                (time) int8 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 -9 2
    air1                   (ti

In [17]:
filename_usv='F:/data/cruise_data/saildrone/baja-2018/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
#ds_usv_tem=xr.open_dataset(filename_usv)
ds_usv = read_saildrone(filename_usv)
#print(ds_usv)
#ds_usv = xr.open_dataset(filename_usv)
#dataset=xr.open_dataset(filename_usv,decode_times=False)
ilen_usv=(len(ds_usv.TEMP_CTD_MEAN['time']))
filled=ds_usv.interpolate_na(dim='time') #fill nan by interpolating
ds_usv10=filled.interp(time=time10)
ds_usvhr=filled.interp(time=btime)
#ds_usv10=filled.resample(time='10Min').mean()  #this code creates 10 averages
#ds_usvhr=filled.resample(time='1H').mean()   #this code creates hourly averages


#dates_usv64=ds_usv.time[0,:].values
#dates_usv=pd.to_datetime(dates_usv64, unit='ns')  #new way to caluclate the time array

AttributeError: 'Dataset' object has no attribute 'LAT'

In [41]:
ds=xr.open_dataset(filename_usv)
ds = ds.sel(trajectory=1002)
ds=ds.rename({'latitude':'lat','longitude':'lon'})
ds=ds.swap_dims({'obs':'time'})
print(ds)
print(ds.time[0:10])


<xarray.Dataset>
Dimensions:               (time: 86839)
Coordinates:
    trajectory            float32 1002.0
  * time                  (time) datetime64[ns] 2018-04-11T18:00:00 ... 2018-06-11T05:57:00
    lat                   (time) float64 ...
    lon                   (time) float64 ...
Data variables:
    SOG                   (time) float64 ...
    COG                   (time) float64 ...
    HDG                   (time) float64 ...
    HDG_WING              (time) float64 ...
    ROLL                  (time) float64 ...
    PITCH                 (time) float64 ...
    WING_ANGLE            (time) float64 ...
    BARO_PRES_MEAN        (time) float64 ...
    BARO_PRES_STDDEV      (time) float64 ...
    TEMP_AIR_MEAN         (time) float64 ...
    TEMP_AIR_STDDEV       (time) float64 ...
    RH_MEAN               (time) float64 ...
    RH_STDDEV             (time) float64 ...
    TEMP_IR_UNCOR_MEAN    (time) float64 ...
    TEMP_IR_UNCOR_STDDEV  (time) float64 ...
    UWND_MEAN   

In [37]:
ds.sel('lat'=90)

SyntaxError: keyword can't be an expression (<ipython-input-37-3e41befdd9b4>, line 1)

In [ ]:
#read in high resolution wind buoy data (10 minute)
fig, (ax1, ax2) = plt.subplots(1,2)

for ibuoy in range(0,6):
    if ibuoy==0:
        sbuoy_id,sdep=['46011','D5']
    if ibuoy==1:
        sbuoy_id,sdep=['46012','D5']
    if ibuoy==2:
        sbuoy_id,sdep=['46028','D6']
    if ibuoy==3:
        sbuoy_id,sdep=['46042','D4']
    if ibuoy==4:
        sbuoy_id,sdep=['46047','D6']
    if ibuoy==5:
        sbuoy_id,sdep=['46086','D5']

    fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201804_'+sdep+'_v0010min_xrformat.nc'
    xr_buoy10=xr.open_dataset(fname)
    fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201805_'+sdep+'_v0010min_xrformat.nc'
    tem=xr.open_dataset(fname)
    xr_buoy10 = xr.concat([xr_buoy10, tem],'time')
    fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201806_'+sdep+'_v0010min_xrformat.nc'
    tem=xr.open_dataset(fname)
    xr_buoy10 = xr.concat([xr_buoy10, tem],'time')
    time10=xr_buoy10.time#read in hourly buoy data
    fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201804_'+sdep+'_v00hrly_xrformat.nc'
    xr_buoy=xr.open_dataset(fname)
    fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201805_'+sdep+'_v00hrly_xrformat.nc'
    tem=xr.open_dataset(fname)
    xr_buoy = xr.concat([xr_buoy, tem],'time')
    fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_'+sbuoy_id+'_201806_'+sdep+'_v00hrly_xrformat.nc'
    tem=xr.open_dataset(fname)
    xr_buoy = xr.concat([xr_buoy, tem],'time')
    time=xr_buoy.time
    #fname='F:/data/cruise_data/saildrone/baja-2018/buoy_data/NDBC_46011_201804_D5_v00hrly_xrformat.nc'
    #xr_buoy=xr.open_dataset(fname)
    cond = xr_buoy.lat > 90
    xr_buoy.lat[cond]=np.nan
    cond = xr_buoy.lon > 90
    xr_buoy.lon[cond]=np.nan
    blat=xr_buoy.lat.mean()
    blon=xr_buoy.lon.mean()
    btime=xr_buoy.time

    #interpolate USV onto buoy 
    ilen_usv=(len(ds_usv.TEMP_CTD_MEAN['time']))
    filled=ds_usv.interpolate_na(dim='time') #fill nan by interpolating
    ds_usv10=filled.interp(time=time10)
    ds_usvhr=filled.interp(time=btime)

    dwin=.25
    dist=((ds_usvhr.LAT-xr_buoy.lat)**2+(ds_usvhr.LON-xr_buoy.lon)**2)**.5
    cond = (dist < dwin)
    subset_buoy=xr_buoy.time[cond]
    tstart=subset_buoy.time.min().data
    tend=subset_buoy.time.max().data

    for ii in range(0,2):
        if ii==0:
            time_start1=tstart
            time_end1=tstart+np.timedelta64(24, 'h')
        if ii==1:
            time_start1=tend-np.timedelta64(24, 'h')
            time_end1=tend
        tstart_buffer=time_start1-np.timedelta64(3, 'h')
        tend_buffer=time_end1+np.timedelta64(3, 'h')
        dwin=.25
        subset=ds_usv.sel(time=slice(time_start1,time_end1))
        lat_usv=subset.LAT
        lon_usv=subset.LON#[0,:]
        time_usv=subset.time#[0,:]
        sst_usv=subset.TEMP_CTD_MEAN#[0,:]
        meets_condition = (lat_usv>blat-dwin) & (lat_usv<blat+dwin) & (lon_usv>blon-dwin) & (lon_usv<blon+dwin) #& time_usv<dt.datetime('2018-04-20')
        #fig, (ax1, ax2) = plt.subplots(1,2)
        ax1.scatter(lon_usv[meets_condition],lat_usv[meets_condition], c=sst_usv[meets_condition], s=5)
        ax1.plot(xr_buoy.lon.sel(time=slice(tstart_buffer,tend_buffer)),xr_buoy.lat.sel(time=slice(tstart_buffer,tend_buffer)),'r.')
        ax1.legend(['Buoy','USV'])
        ax2.plot(subset.time[meets_condition],subset.TEMP_CTD_MEAN[meets_condition]-273.15,'.')
        ax2.plot(xr_buoy.time,xr_buoy.sst1-273.15)
        ax2.set_xlim(tstart_buffer,tend_buffer)
        ax2.set_ylim(11,14.5)
        # rotate and align the tick labels so they look better
        fig.autofmt_xdate()
        fig_fname='F:/data/cruise_data/saildrone/baja-2018/buoy_loc_image_25km'+sbuoy_id+str(ii)+'.png'
        fig.savefig(fig_fname, transparent=False, format='png')
        plt.clf()

        dwin=.02
        subset=ds_usv.sel(time=slice(time_start1,time_end1))
        lat_usv=subset.LAT
        lon_usv=subset.LON#[0,:]
        time_usv=subset.time#[0,:]
        sst_usv=subset.TEMP_CTD_MEAN#[0,:]
        meets_condition = (lat_usv>blat-dwin) & (lat_usv<blat+dwin) & (lon_usv>blon-dwin) & (lon_usv<blon+dwin) #& time_usv<dt.datetime('2018-04-20')
        #fig, (ax1, ax2) = plt.subplots(1,2)
        ax1.scatter(lon_usv[meets_condition],lat_usv[meets_condition], c=sst_usv[meets_condition], s=5)
        ax1.plot(xr_buoy.lon.sel(time=slice(tstart_buffer,tend_buffer)),xr_buoy.lat.sel(time=slice(tstart_buffer,tend_buffer)),'r.')
        ax1.legend(['Buoy','USV'])
        ax2.plot(subset.time[meets_condition],subset.TEMP_CTD_MEAN[meets_condition]-273.15,'.')
        ax2.plot(xr_buoy.time,xr_buoy.sst1-273.15)
        ax2.set_xlim(tstart_buffer,tend_buffer)
        ax2.set_ylim(11,14.5)
        # rotate and align the tick labels so they look better
        fig.autofmt_xdate()
        fig_fname='F:/data/cruise_data/saildrone/baja-2018/buoy_loc_image2km'+sbuoy_id+str(ii)+'.png'
        fig.savefig(fig_fname, transparent=False, format='png')
        plt.clf()

        #now i only use part of buoy location that is for this time
        dwin=.01
        subset=ds_usv.sel(time=slice(time_start1,time_end1))
        lat_usv=subset.LAT
        lon_usv=subset.LON#[0,:]
        time_usv=subset.time#[0,:]
        sst_usv=subset.TEMP_CTD_MEAN#[0,:]
        blon_subset=xr_buoy.lon.sel(time=slice(time_start1,time_end1)).mean()
        blat_subset=xr_buoy.lat.sel(time=slice(time_start1,time_end1)).mean()
        meets_condition = (lat_usv>blat_subset-dwin) & (lat_usv<blat_subset+dwin) & (lon_usv>blon_subset-dwin) & (lon_usv<blon_subset+dwin) #& time_usv<dt.datetime('2018-04-20')
        #fig, (ax1, ax2) = plt.subplots(1,2)
        ax1.scatter(lon_usv[meets_condition],lat_usv[meets_condition], c=sst_usv[meets_condition], s=5)
        ax1.plot(xr_buoy.lon.sel(time=slice(time_start1,time_end1)),xr_buoy.lat.sel(time=slice(time_start1,time_end1)),'r.')
        ax1.legend(['Buoy','USV'])
        ax2.plot(subset.time[meets_condition],subset.TEMP_CTD_MEAN[meets_condition]-273.15,'.')
        ax2.plot(xr_buoy.time,xr_buoy.sst1-273.15)
        ax2.set_xlim(time_start1,time_end1)
        ax2.set_ylim(11,14.5)
        # rotate and align the tick labels so they look better
        fig.autofmt_xdate()
        fig_fname='F:/data/cruise_data/saildrone/baja-2018/buoy_loc_image1km'+sbuoy_id+str(ii)+'.png'
        fig.savefig(fig_fname, transparent=False, format='png')
        plt.clf()

    dwin=.06
    bair=xr_buoy.air1
    #print(xr_buoy.lat.shape)
    #print(ds_usvhr.LAT.shape)
    dist=((ds_usvhr.LAT-xr_buoy.lat)**2+(ds_usvhr.LON-xr_buoy.lon)**2)**.5
    cond = (dist < dwin)
    subset_usv=ds_usvhr.TEMP_CTD_MEAN[cond]
    subset_buoy=xr_buoy.sst1[cond]
    print(sbuoy_id,subset_usv.shape)
    print('sst:',(subset_usv-subset_buoy).mean().data,(subset_usv-subset_buoy).std().data)
    subset_usv=ds_usvhr.TEMP_AIR_MEAN[cond]
    subset_buoy=xr_buoy.air1[cond]-273.15
    print('air:',(subset_usv-subset_buoy).mean().data,(subset_usv-subset_buoy).std().data)
    subset_usv=ds_usvhr.wind_speed[cond]
    x2=xr_buoy.where((xr_buoy<10000)) #mask bad values
    if ibuoy==2 or ibuoy==1:
        subset_buoy=x2.wnd2[cond]
    else:
        subset_buoy=x2.wnd1[cond]       
    print('wnd:',(subset_usv-subset_buoy).mean().data,(subset_usv-subset_buoy).std().data)
    subset_usv=abs(ds_usvhr.wind_direction[cond])*360
    x2=xr_buoy.where((xr_buoy>-10000))  #mask bad values
    if ibuoy==2 or ibuoy==1:
        subset_buoy=x2.wdir2[cond]
    else:
        subset_buoy=x2.wdir1[cond]       
    #subset_buoy=x2.wdir1[cond]
    print('wdir:',(subset_usv-subset_buoy).mean().data,(subset_usv-subset_buoy).std().data)

In [ ]:
dwin=.05
bair=xr_buoy.air1
dist=((ds_usvhr.LAT-xr_buoy.lat)**2+(ds_usvhr.LON-xr_buoy.lon)**2)**.5
cond = (dist < dwin)
subset_usv=ds_usvhr.TEMP_CTD_MEAN[cond]
subset_buoy=xr_buoy.sst1[cond]
subset_usv.shape

In [ ]:
#ds_usv10=ds_usv.interp(time=time10)
#ds_usvhr=ds_usv.interp(time=btime)


In [ ]:
print(abs(subset_usv.data*360))
print(subset_buoy.data)

In [ ]:
xr_buoy
#subset=ds_usvhr.sel(time=slice(xr_buoy.time.min(),xr_buoy.time.max()))
#subset_buoy=xr_buoy.sel(time=slice(ds_usvhr.time.min(),ds_usvhr.time.max()))
#print(subset.time)
#print(subset_buoy.time)

In [ ]:
ds_usvhr.TEMP_AIR_MEAN.plot()

In [ ]:
ds_usv_tem.UWND_MEAN.plot()

In [ ]:
dwin=.02
subset=ds_usv.sel(time=slice('2018-05-31','2018-05-31 12:00:00'))
lat_usv=subset.LAT
lon_usv=subset.LON#[0,:]
time_usv=subset.time#[0,:]
sst_usv=subset.TEMP_CTD_MEAN#[0,:]
meets_condition = (lat_usv>blat-dwin) & (lat_usv<blat+dwin) & (lon_usv>blon-dwin) & (lon_usv<blon+dwin) #& time_usv<dt.datetime('2018-04-20')
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.scatter(lon_usv[meets_condition],lat_usv[meets_condition], c=sst_usv[meets_condition], s=5)
ax2.plot(subset.time[meets_condition],subset.TEMP_CTD_MEAN[meets_condition]-273.15,'.')
ax2.plot(xr_buoy.time,xr_buoy.sst1-273.15)
ax2.set_xlim('2018-05-31','2018-05-31 12:00:00')
ax2.set_ylim(11.5,12.5)

In [ ]:
bair

In [ ]:
print(xr_buoy.lat.mean())
print(time_usv.dt.hour)

In [ ]:
time_usv_pd.year

In [ ]:
#ds_usv.TEMP_CTD_MEAN.resample(time="10-min").mean()
#ds_usv.sel(time=slice('2018-04-20','2018-04-25'))